In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import plotly.graph_objects as go

from collections import namedtuple

np.random.seed(7)

## Байесовские А/Б-тесты

Показана реализация А/Б-тестов. Рассмотрено использование байесовского моделирования для сравнения конверсий и средних. Дополнительно обсуждаются множественные сравнения и транзакционная выручка на пользователя.

https://stepik.org/course/249642/promo

### Материалы курса 

Байесовская оценка А/Б-тестов:
- код: https://github.com/andrewbrdk/Bayesian-AB-Testing
- видео: https://www.youtube.com/playlist?list=PLqgtGAeapsOPpV0FqeXEpWosHBW8ZebYl  

Реализация А/Б-тестов:
- код: https://github.com/andrewbrdk/AB-Testing-Implementation


Рекомендации:  
Теория вероятностей:  
- J. Tsitsiklis, P. Jaillet, Introduction to Probability ([видео](https://www.youtube.com/playlist?list=PLUl4u3cNGP60hI9ATjSFgLZpbNJ7myAg6), [материалы](https://ocw.mit.edu/courses/res-6-012-introduction-to-probability-spring-2018/))

Байесовское моделирование:  
- B. Lambert, A Student’s Guide to Bayesian Statistics ([книга](https://www.amazon.co.uk/Students-Guide-Bayesian-Statistics/dp/1473916364), [материалы](https://study.sagepub.com/lambert))  
- R. McElreath, Statistical Rethinking: A Bayesian Course with Examples in R and STAN ([книга](https://www.routledge.com/Statistical-Rethinking-A-Bayesian-Course-with-Examples-in-R-and-STAN/McElreath/p/book/9780367139919), [видео](https://www.youtube.com/playlist?list=PLDcUM9US4XdPz-KxHM4XHt7uUVGWWVSus), [материалы](https://github.com/rmcelreath/stat_rethinking_2024))  

Платформы А/Б-тестов:  
- [GrowthBook](https://www.growthbook.io/)  


## А/Б-тесты

### После видео

Действия пользователей в продукте и целевые метрики можно описывать случайными величинами. Различия их распределений между группами А/Б-теста при прочих равных условиях (внешние факторы, состав аудитории) можно объяснять версиями продукта. Точные распределения неизвестны. Иногда их удается приблизить аналитическими моделями, но даже в этом случае остаются неизвестны точные параметры. В эксперименте собирается выборка из распределений. По выборке нужно построить оценки точных распределений, их свойств и выбрать лучший вариант.

Пусть $X_1,X_2,\dots,X_n$ - независимые одинаково распределенные случайные величины. При большом количестве данных по [закону больших чисел](https://ru.wikipedia.org/wiki/%D0%97%D0%B0%D0%BA%D0%BE%D0%BD_%D0%B1%D0%BE%D0%BB%D1%8C%D1%88%D0%B8%D1%85_%D1%87%D0%B8%D1%81%D0%B5%D0%BB#%D0%91%D0%BE%D1%80%D0%B5%D0%BB%D0%B5%D0%B2%D1%81%D0%BA%D0%B8%D0%B9_%D0%B7%D0%B0%D0%BA%D0%BE%D0%BD_%D0%B1%D0%BE%D0%BB%D1%8C%D1%88%D0%B8%D1%85_%D1%87%D0%B8%D1%81%D0%B5%D0%BB) распределение выборки близко точному распределению. [Выборочное среднее](https://ru.wikipedia.org/wiki/%D0%92%D1%8B%D0%B1%D0%BE%D1%80%D0%BE%D1%87%D0%BD%D0%BE%D0%B5_%D1%81%D1%80%D0%B5%D0%B4%D0%BD%D0%B5%D0%B5) $\overline{X}_n$ приближает точное среднее $E[X]$, [выборочная дисперсия](https://ru.wikipedia.org/wiki/%D0%92%D1%8B%D0%B1%D0%BE%D1%80%D0%BE%D1%87%D0%BD%D0%B0%D1%8F_%D0%B4%D0%B8%D1%81%D0%BF%D0%B5%D1%80%D1%81%D0%B8%D1%8F) $\sigma_n^2$ - точную дисперсию $\sigma^2$ .

...

In [ ]:
import plotly.graph_objects as go

p_A = 0.2

values = [0, 1]
probs_A = [1 - p_A, p_A]
width = 0.2

fig = go.Figure()
fig.add_trace(go.Bar(x=values, y=probs_A, name=f"A, p={p_A}", width=width, marker_color='black'))
fig.update_layout(
    title="Распределение Бернулли",
    #xaxis_title="",
    yaxis_title="Вероятность",
    barmode="group",
    yaxis_range=[0, 1.3],
    bargap=0.6,
    template="plotly_white",
)

fig.show()
#fig.write_image("./stepik_Bern_dist.png", scale=2)

### Задача: 2 величины Бернулли

Пусть есть две случайных величины с распределением Бернулли. Вероятность выпадения единицы в одной $p_A=0.2$, в другой $p_B=0.25$. Отметьте верные утверждения:

In [ ]:
import plotly.graph_objects as go

p_A = 0.2
p_B = 0.25

values = [0, 1]
probs_A = [1 - p_A, p_A]
probs_B = [1 - p_B, p_B]
width = 0.2

fig = go.Figure()
fig.add_trace(go.Bar(x=values, y=probs_A, name=f"A, p={p_A}", width=width, marker_color='black', opacity=0.3))
fig.add_trace(go.Bar(x=values, y=probs_B, name=f"B, p={p_B}", width=width, marker_color='black'))
fig.update_layout(
    title="Две случайных величины с распределением Бернулли",
    #xaxis_title="",
    yaxis_title="Вероятность",
    barmode="group",
    yaxis_range=[0, 1.5],
    bargap=0.6,
    template="plotly_white",
)

fig.show()
#fig.write_image("./stepik_2Bern_dist.png", scale=2)

--Среднее Б больше А.  
E[x] = p, p_B = 0.25, p_A = 0.2

--Дисперсия Б больше А.  
sigma^2= p * (1 - p), sigma^2_A= 0.16, sigma^2_B = 0.1875

--По мере набора данных доля единиц в выборках будет уменьшаться.  
По закону больших чисел распределение в выборке будет приближаться к точному распределению. Т.е. доли единиц будут постоянными 0.2 для А и 0.25 для Б.

--По мере набора данных выборочные средние будут приближаться к точным средним.  
Выборочное среднее - состоятельная оценка точного среднего.

--По мере набора данных выборочные дисперсии будут стремиться к нулю.  
Выборочные дисперсии будут приближаться к точным дисперсиям.

--По мере набора данных стандартные ошибки средних будут стремиться к нулю.  
Стандартная ошибка среднего - отношение стандартного отклонения точного распределения к корню из размера выборки. Дисперсии постоянны, выборка растет - отношение стремится к нулю.

--По мере набора данных разность выборочных средних будет стремиться к нулю.   
Выборочные средние стремятся к точным средним. Разность выборочных средних будет стремиться к разности точных средних. Она отлична от нуля.

--По мере набора данных дисперсия разности выборочных средних будет стремиться к нулю.  
Дисперсии выборочных средних в группах стремятся к нулю, дисперсия разности также будет стремиться к нулю.

--Вероятность разности выборочных средних больше 0 не превзойдет 0.8 при любом количестве данных.  
Разность выборочных средних стремится к точной разности, дисперсия разности выборочных средних стремится к нулю. Вероятность разности выборочных средних больше 0 будет стремиться к 1.

## Реализация А/Б-тестов

In [ ]:
def good_enough(probs, N):
  counts = [0] * len(probs)
  for _ in range(N):
    g = assign_group(probs)
    counts[g] += 1
  for c, p in zip(counts, probs):
    samp_p = c / N
    stderr = sqrt(p * (1-p)) / sqrt(N)
    if samp_p < p - 5 * stderr or samp_p > p + 5 * stderr:
      return False
  return True

In [ ]:
import random

def assign_group(probs: list[float]) -> int:
    r = random.random()
    cumulative = 0.0
    for i, p in enumerate(probs):
        cumulative += p
        if r < cumulative:
            return i
    return len(probs) - 1

s = '0.2 0.5 0.3'
probs = [float(x.strip()) for x in s.split()]
N = 100_000
counts = [0] * len(probs)
for _ in range(N):
  g = assign_group(probs)
counts[g] += 1
print(" ".join(str(x) for x in counts))

In [ ]:
import math

p = 0.1
N = 100000
sem = math.sqrt(p * (1 - p) / N)
print(sem)

In [ ]:
int(random.uniform(0, len(probs)))

In [ ]:
import random

def assign_group(probs: list[float]) -> int:
    r = random.random()
    cumulative = 0.0
    for i, p in enumerate(probs):
        cumulative += p
        if r < cumulative:
            return i
    return len(probs) - 1

probs = [float(x.strip()) for x in input().split()]
N = 100_000
counts = [0] * len(probs)
for _ in range(N):
    g = assign_group(probs)
    counts[g] += 1
print(" ".join(str(g) for g in counts))

In [ ]:
from math import sqrt

def check(reply, clue):
    N = 100_000
    probs = clue
    counts = [int(x.strip()) for x in reply.split()]
    for c, p in zip(counts, probs):
        samp_p = c / N
        stderr = sqrt(p * (1-p) / N)
        if samp_p < p - 5 * stderr or samp_p > p + 5 * stderr:
            return False
    return True

check("10062 29707 60231", (0.1, 0.3, 0.6))
check("49867 50133", (0.5, 0.5))

In [ ]:
def generate():
    w = [
        (0.5, 0.5),
        (0.9, 0.1),
        (0.3, 0.5, 0.2),
        (0.25, 0.25, 0.3, 0.2),
        (0.4, 0.2, 0.15, 0.15, 0.1)
    ]
    tests=[(' '.join(str(i) for i in x) + '\n', x) for x in w] 
    return tests

generate()

In [ ]:
import random

def assign_group(probs: list[float]) -> int:
    r = random.random()
    cumulative = 0.0
    for i, p in enumerate(probs):
        cumulative += p
        if r < cumulative:
            return i
    return len(probs) - 1

probs = [float(x.strip()) for x in input().split()]
N = 100_000
counts = [0] * len(probs)
for _ in range(N):
    g = assign_group(probs)
    counts[g] += 1
print(" ".join(str(g) for g in counts))

In [ ]:
from math import sqrt

import random

def assign_group(probs: list[float]) -> int:
    return random.choice([0, 1])

def check(probs):
    N = 100_000
    counts = [0] * len(probs)
    for _ in range(N):
        g = assign_group(probs)
        counts[g] += 1
    for c, p in zip(counts, probs):
        samp_p = c / N
        stderr = sqrt(p * (1-p) / N)
        if samp_p < p - 5 * stderr or samp_p > p + 5 * stderr:
            return False
    return True

probs = [0.5, 0.5, 0.3]
check(probs)

In [ ]:
random.choices(range(len(probs)), probs)[0]

In [ ]:
import numpy as np

probs = [0.5, 0.5]
np.random.choice(list(range(len(probs))), p=probs)

## Конверсии

In [ ]:
na = 10000
sa = 1000
nb = 10000
sb = 1100

d = 0.01

p_dist_a = stats.beta(a=sa+1, b=na-sa+1)
p_dist_b = stats.beta(a=sb+1, b=nb-sb+1)

approx_diff_dist = stats.norm(loc=p_dist_b.mean() - p_dist_a.mean(), 
                              scale=np.sqrt(p_dist_b.std()**2 + p_dist_a.std()**2))
dist_p_b_gt_a = 1 - approx_diff_dist.cdf(d)

npost = 100000
samp_a = p_dist_a.rvs(size=npost)
samp_b = p_dist_b.rvs(size=npost)
samp_p_b_gt_a = np.sum(samp_b - samp_a > d) / npost


xaxis_max = 0.2
x = np.linspace(0, xaxis_max, 1000)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=p_dist_a.pdf(x), line_color='black', name='А'))
fig.add_trace(go.Scatter(x=x, y=p_dist_b.pdf(x), line_color='black', opacity=0.3, name='Б'))
fig.update_layout(title='Апостериорные распределения',
                  xaxis_title='$p$',
                  yaxis_title='Плотность вероятности',
                  xaxis_range=[0, xaxis_max],
                  hovermode="x",
                  height=500)
fig.show()
#fig.write_image("./figs/ch2_conv_cmp_example.png", scale=2)
#Апостериорные распределения конверсий в обеих группах задаются бета-распределениями.

x = np.linspace(-0.3, 0.3, 1000)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=approx_diff_dist.pdf(x), 
                         line_color='black', name='$\mbox{Аналитическое приближение}$'))
fig.add_trace(go.Histogram(x=samp_b - samp_a, histnorm='probability density', 
                           name='$\mbox{Разность апостериорных выборок}$', nbinsx=500,
                           marker_color='black', opacity=0.3))
fig.add_trace(go.Scatter(x=[0, 0], y=[0, max(approx_diff_dist.pdf(x))*1.05], 
                         line_color='black', mode='lines', line_dash='dash', showlegend=False))
fig.update_layout(title='$p_B - p_A$',
                  xaxis_title='$x$',
                  yaxis_title='Плотность вероятности',
                  xaxis_range=[-0.1, 0.1],
                  hovermode="x",
                  height=500)
fig.show()
#fig.write_image("./figs/ch2_conv_cmp_diff.png", scale=2)
#Конверсия группы Б выше А с вероятностью 77%.

print(f"P(p_b > p_a) diff dist: {dist_p_b_gt_a}")
print(f"P(p_b > p_a) post samples: {samp_p_b_gt_a}")

In [ ]:
(0.93 - 0.89) / 0.93

In [ ]:
na = 10000
sa = 1000
nb = 10000
sb = 1100

d = 0.01

p_dist_a = stats.beta(a=sa+1, b=na-sa+1)
p_dist_b = stats.beta(a=sb+1, b=nb-sb+1)

approx_diff_dist = stats.norm(loc=p_dist_b.mean() - p_dist_a.mean(), 
                              scale=np.sqrt(p_dist_b.std()**2 + p_dist_a.std()**2))
approx_diff_p = 1 - approx_diff_dist.cdf(d)

npost = 100000
samp_a = p_dist_a.rvs(size=npost)
samp_b = p_dist_b.rvs(size=npost)
samp_diff_p = np.sum(samp_b - samp_a > d) / npost

print(f"P(p_b - p_a > {d}) diff dist: {approx_diff_p}")
print(f"P(p_b - p_a > {d}) post samples: {samp_diff_p}")

# P(p_b - p_a > 0.01) diff dist: 0.49981600299255935
# P(p_b - p_a > 0.01) post samples: 0.50065

In [ ]:
na = 10000
sa = 1000
nb = 10000
sb = 1100

rd = 0.01

p_dist_a = stats.beta(a=sa+1, b=na-sa+1)
p_dist_b = stats.beta(a=sb+1, b=nb-sb+1)

rd_mu = (p_dist_b.mean() - p_dist_a.mean()) / p_dist_a.mean()
rd_s = np.abs(p_dist_b.mean() / p_dist_a.mean()) * np.sqrt((p_dist_a.std() / p_dist_a.mean())**2 + (p_dist_b.std() / p_dist_b.mean())**2)

approx_reldiff_dist = stats.norm(loc=rd_mu, scale=rd_s)
approx_reldiff_p = 1 - approx_reldiff_dist.cdf(rd)

npost = 100000
samp_a = p_dist_a.rvs(size=npost)
samp_b = p_dist_b.rvs(size=npost)
samp_reldiff_p = np.sum((samp_b - samp_a)/ samp_a > rd) / npost

print(f"P((p_b-p_a)/p_a > {d}) rel diff dist: {approx_reldiff_p}")
print(f"P((p_b-p_a)/p_a > {d}) post samples: {samp_reldiff_p}")

# P((p_b-p_a)/p_a > 0.01) diff dist: 0.97604762812634
# P((p_b-p_a)/p_a > 0.01) post samples: 0.98068

In [ ]:
na = 10000
sa = 1000
nb = 10000
sb = 1100

p_dist_a = stats.beta(a=sa+1, b=na-sa+1)
p_dist_b = stats.beta(a=sb+1, b=nb-sb+1)

approx_diff_dist = stats.norm(loc=p_dist_b.mean() - p_dist_a.mean(), 
                              scale=np.sqrt(p_dist_b.std()**2 + p_dist_a.std()**2))
approx_diff_p = 1 - approx_diff_dist.cdf(0)

npost = 100000
samp_a = p_dist_a.rvs(size=npost)
samp_b = p_dist_b.rvs(size=npost)
samp_diff_p = np.sum(samp_b - samp_a > 0) / npost

print(f"P(p_b > p_a) diff dist: {approx_diff_p}")
print(f"P(p_b > p_a) post samples: {samp_diff_p}")

# P(p_b > p_a) diff dist: 0.9894463764471134
# P(p_b > p_a) post samples: 0.98956

In [ ]:
na = 10000
sa = 1000
nb = 10000
sb = 1100

p_dist_a = stats.beta(a=sa+1, b=na-sa+1)
p_dist_b = stats.beta(a=sb+1, b=nb-sb+1)

approx_diff_dist = stats.norm(loc=p_dist_b.mean() - p_dist_a.mean(), 
                              scale=np.sqrt(p_dist_b.std()**2 + p_dist_a.std()**2))
approx_diff_p = 1 - approx_diff_dist.cdf(0)

npost = 100000
samp_a = p_dist_a.rvs(size=npost)
samp_b = p_dist_b.rvs(size=npost)
samp_diff_p = np.sum(samp_b > samp_a) / npost

print("Without prior data:")
print(f"P(p_b > p_a) diff dist: {approx_diff_p}")
print(f"P(p_b > p_a) post samples: {samp_diff_p}")
print()

N = 50000
Ns = 5200
alpha = Ns
beta = N - Ns

p_dist_a = stats.beta(a=alpha+sa, b=beta+na-sa)
p_dist_b = stats.beta(a=alpha+sb, b=beta+nb-sb)

approx_diff_dist = stats.norm(loc=p_dist_b.mean() - p_dist_a.mean(), 
                              scale=np.sqrt(p_dist_b.std()**2 + p_dist_a.std()**2))
approx_diff_p = 1 - approx_diff_dist.cdf(0)

npost = 100000
samp_a = p_dist_a.rvs(size=npost)
samp_b = p_dist_b.rvs(size=npost)
samp_diff_p = np.sum(samp_b > samp_a) / npost

print("With prior data:")
print(f"P(p_b > p_a) diff dist: {approx_diff_p}")
print(f"P(p_b > p_a) post samples: {samp_diff_p}")

#Without prior data:
#P(p_b > p_a) diff dist: 0.9894463764471134
#P(p_b > p_a) post samples: 0.98873
#
#With prior data:
#P(p_b > p_a) diff dist: 0.8276732342040869
#P(p_b > p_a) post samples: 0.82686

In [ ]:
import numpy as np
from scipy import stats

N = 10000
sa = 1030
sb = 1050
sc = 1100

p_dist_a = stats.beta(a=sa+1, b=N-sa+1)
p_dist_b = stats.beta(a=sb+1, b=N-sb+1)
p_dist_c = stats.beta(a=sc+1, b=N-sc+1)

npost = 500000
samp_a = p_dist_a.rvs(size=npost)
samp_b = p_dist_b.rvs(size=npost)
samp_c = p_dist_c.rvs(size=npost)

pc_gt_papb = np.sum((samp_c > samp_a) & (samp_c > samp_b)) / npost
pc_gt_pa = np.sum(samp_c > samp_a) / npost
pc_gt_pb = np.sum(samp_c > samp_b) / npost

print(f"P(p_c > p_a & p_c > p_b): {pc_gt_papb}")
print(f"P(p_c > p_a): {pc_gt_pa}")
print(f"P(p_c > p_b): {pc_gt_pb}")

#P(p_c > p_a & p_c > p_b): 0.842206
#P(p_c > p_a): 0.945612
#P(p_c > p_b): 0.872644

О Байесовском моделировании можно думать как об обновлении вероятностей гипотез по мере поступления новых данных.
Формулируются гипотезы $H_i$. Для них задаются априорные вероятности $P(H_i)$. По мере поступления данных $x_i$ для каждой гипотезы считается вероятность $P(H_i \cap x_1 \dots \cap x_N)$ - функции правдоподобия. Апостериорная вероятность $P(\mathcal{H} | \mathcal{D})$ будет равна вероятности получить данные в рамках гипотезы нормированной на вероятности в рамках всех гипотез

$$
P(\mathcal{H}_i | \mathcal{D}) = \frac{P(\mathcal{D} | \mathcal{H}_i) P(\mathcal{H}_i)}{\sum_j P(\mathcal{D} | \mathcal{H}_j) P(\mathcal{H}_j)}
$$

<center>
<img src="../figs/bayes_update.png" alt="bayes_update" width="500"/>
<em>
<br/>
Обновление вероятностей гипотез по мере поступления данных.
</em>
</center>

### Задача: выбор гипотез

На дашборде вы видите падение конверсии из созданного заказа в оплату по сравнению с предыдущими днями.

По прошлому опыту аномалии в отчетах чаще всего связаны проблемами обновления данных. Дальше баги после релизов на проде. Либо в событиях, либо в основной функциональности. Бывают специфические проблемы отдельных сегментов. Численные априорные вероятности записаны в таблице ниже.

Падение заметно на глаз на фоне обычных колебаний метрики. Все обновления данных отработали штатно, в других отчетах падений не видно. Расчет конверсии не завязан на события, а считается по данным с сервера. Релизов вчера не было.  С учетом этих данных вы оцениваете вероятность аномалии в рамках каждой гипотезы (правдоподобия). 

Посчитайте вероятность наиболее вероятной гипотезы. В ответе укажите вероятность наиболее вероятной гипотезы, вероятность переведите в проценты и округлите до целых.




| Гипотеза H | Априорная вероятность P(H) | Правдоподобие P(D \| H) |
|------------|------------|----------------------|
| Баг в событиях | 0.40 | 0.25 |
| Ошибка обновления отчетов | 0.20 | 0.10 |
| Баг на проде | 0.25 | 0.20 |
| Изменение в одном из сегментов | 0.15 | 0.70 |
| Другие причины | ? | ? |


Вы видите резкий рост уникальных пользователей в отчетах.  
Вы хотите понять, в чем дело.  
Предполагаете причины: проблема в трекинге, проблема в дашборде, баг на проде, проблема в одном из сегментов. 

Вам пришло уведомление о снижении конверсии из созданного заказа в оплату. 

Вы видите падение конверсии из созданного заказа в оплату. 


| Hypothesis | Description | Prior P(H) | Likelihood P(E \| H) |
|-----------|------------|------------|----------------------|
| H₁ | Tracking instrumentation bug | 0.40 | 0.25 |
| H₂ | Backend service degradation | 0.20 | 0.10 |
| H₃ | Marketing campaign ended | 0.25 | 0.20 |
| H₄ | Segment-level behavior change | 0.15 | 0.70 |

### Задача: базовый процент

Ботов в трафике 3%.  
Они распознаются с вероятностью 99%.  
Реальные пользователи помечаются как боты с вероятностью 4%.  
Сколько среди отмеченных ботами реальных пользователей?  
Ответ округлите до целых. 

In [ ]:
#           | bot, 3% | real, 97%
# marked    | 99      |  4
# nonmarked |         |  

p_bot = 0.03
p_bot_mark = 0.99
p_real = 1 - p_bot
p_real_mark = 0.04

p_real * p_real_mark / (p_bot * p_bot_mark + p_real * p_real_mark)

Подозрительных транзакций 2%.  
Они помечаются с вероятностью 99%.  
Обычные транзакции помечаются подозрительными с вероятностью 3%.  
Сколько среди отмеченных транзакций обычных?  
Ответ округлите до целых. 

In [ ]:
#           | susp, 2% | common
# marked, % | 99       |  3
# nonmarked |          |  

p_susp = 0.02
p_susp_mark = 0.99
p_com_mark = 0.03
p_com = 1 - p_susp

p_com * p_com_mark / (p_susp * p_susp_mark + p_com * p_com_mark)

### Задача: качество гипотез

Кроме вероятности выбрать лучшую группу в экспериментах важно качество гипотез.

Изменение может улучшить продукт с вероятностью $h$ или ухудшить с вероятностью $1-h$.  
Вы распознаете лучшую группу в эксперименте с вероятностью $p$.  
Всего провели $N$ экспериментов.  
Посчитайте количество внедренных улучшений и ухудшений для $N=50, p = 95\%, h = 10\%$.  
В ответе запишите количество улучшений и ухудшений через пробел, округлите до целых.

<center>
<img src="../figs/bayes_better_worse_guess.png" alt="bayes_better_worse_guess" width="500"/>
</center>

In [ ]:
N = 50
p = 0.95
h = 0.1

S = N * h * p
print('S', S)

L = N * (1-h) * (1-p)
print('L', L)

Улучшения определяются качеством гипотез.  
Можно добиться такого же количества улучшений с меньшим количеством экспериментов, но с большим качеством.

In [ ]:
N = 10
p = 0.95
h = 0.3

S = N * h * p
print('S', S)

L = N * (1-h) * (1-p)
print('L', L)

### Средняя выручка на пользователя - ЦПТ vs Lognorm

В выручке на пользователя $P_{пользователи}(x)$ удобно выделить выручку на платящего $P_{платящие}(x)$. При конверсии в оплату $p$ распределение ненулевой выручки на пользователя $p P_{платящие}(x)$, с вероятностью $1-p$ выручка нулевая.

$$
P_{пользователи}(x) = 
\begin{cases}
1-p, \, x = 0
\\
p P_{платящие}(x), \, x > 0
\end{cases}
$$

Для транзакционных сервисов, в частности маркетплейсов, выручку на платящего часто можно моделировать логнормальным распределением. Случайная величина логнормальная $X \sim \text{Lognormal}(\mu, s^2)$, если логарифм распределен нормально $\ln(X) \sim \text{Norm}(\mu, s^2)$.

Сопряженное априорное распределение к логнормальной функции правдоподобия $P(\mathcal{D} | \mathcal{H}) = \text{Lognorm}(x | \mu, s^2)$ строится аналогично нормальному распределению. Для упрощенной модели с одним параметром $\mu$ и фиксированным $s$ сопряженное априорное распределение нормальное $P(\mu) = \text{Norm}(\mu | \mu_0, \sigma_0^2)$ с параметрами $\mu_0$, $\sigma_0$. Апостериорное распределение нормальное $P(\mu | \mathcal{D}) = \text{Norm}(\mu | \mu_N, \sigma_N^2)$ с обновленными параметрами $\mu_N$, $\sigma_N$. В $\mu_N$ суммируются логарифмы точек выборки.

$$
\begin{split}
P(\mathcal{D} | \mathcal{H}) & = \text{Lognorm}(x | \mu, s^2) = 
\frac{1}{x \sqrt{2 \pi s^2}} e^{-\tfrac{(\ln x - \mu)^2}{2 s^2}}
\\
P(\mathcal{H}) & = \text{Norm}(\mu | \mu_0, \sigma_0^2) = 
\frac{1}{\sqrt{2 \pi \sigma_{0}^2}} e^{-\tfrac{(\mu-\mu_0)^2}{2 \sigma_{0}^2}} 
\\
P(\mathcal{H} | \mathcal{D}) 
& = \text{Norm}(\mu | \mu_N, \sigma_N^2),
\quad
\sigma_N^2 = \frac{\sigma_0^2 s^2}{s^2 + N \sigma_0^2},
\quad
\mu_N = \mu_0 \frac{\sigma_N^2}{\sigma_0^2} + \frac{\sigma_N^2}{s^2} \sum_i^N \ln x_i
\end{split}
$$

Среднее логнормального распределения
$$
E[x] = e^{\mu + s^2/2}
$$

Зафиксировать распределение выручки на пользователя как p * lognorm.  
Сгенерировать выборку.  
Сравнить среднюю выручку на пользователя по ЦПТ с точной. 

In [ ]:
def exact_rev_per_user_rvs(p, mu, s, nsamples):
    conv = stats.bernoulli.rvs(p=p, size=nsamples)
    rev = stats.lognorm.rvs(s=s, scale=np.exp(mu), size=nsamples)
    return conv * rev

#p = 
s = 1
mu = 8
nsample = 5000
exact_dist = stats.lognorm(s=s, scale=np.exp(mu))
data = exact_dist.rvs(nsample)

xaxis_max=30000
x = np.linspace(0, xaxis_max, 10000)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=exact_dist.pdf(x), line_dash='solid', line_color='black', name='Точное распределение'))
fig.add_trace(go.Scatter(x=[np.log(exact_dist.mean()), np.log(exact_dist.mean())], y=[0, max(exact_dist.pdf(x))*1.05], 
                         line_color='black', mode='lines', line_dash='dash', name='Логарифм точного среднего'))
fig.add_trace(go.Scatter(x=[exact_dist.mean(), exact_dist.mean()], y=[0, max(exact_dist.pdf(x))*1.05], 
                         line_color='black', mode='lines', line_dash='dash', name='Точное среднее'))
fig.update_layout(title='$\mbox{Lognorm распределение } x$',
                  xaxis_title='$x$',
                  yaxis_title='Плотность вероятности',
                  #xaxis_range=[0, 10],
                  barmode='overlay',
                  hovermode="x",
                  height=500)                  
fig.show()

Можно не усложнять все логнормальными распределениями.  
Попробовать просто распределение Бернулли * 5000, т.е. {0, 5000}. 

2 группы с распр. Бернулли.  
A: p, x,  
B: p \* 0.95, x \* 1.05

Средние:  
A: p \* x   
B: p \* x \* 0.95 \* 1.05  

Сгенерировать выборки.  
По выборкам посчитать относительную разность.  
Сравнить с реальной.  

In [ ]:
pa = 0.1
a = 5000
pb = pa * 0.95
b = a * 1.07



In [ ]:
na = 10000
sa = 1000
nb = 10000
sb = 1100

p_dist_a = stats.beta(a=sa+1, b=na-sa+1)
p_dist_b = stats.beta(a=sb+1, b=nb-sb+1)

approx_diff_dist = stats.norm(loc=p_dist_b.mean() - p_dist_a.mean(), 
                              scale=np.sqrt(p_dist_b.std()**2 + p_dist_a.std()**2))
approx_diff_p = 1 - approx_diff_dist.cdf(0)

npost = 100000
samp_a = p_dist_a.rvs(size=npost)
samp_b = p_dist_b.rvs(size=npost)
samp_diff_p = np.sum(samp_b - samp_a > 0) / npost

print(f"P(p_b > p_a) diff dist: {approx_diff_p}")
print(f"P(p_b > p_a) post samples: {samp_diff_p}")

# P(p_b > p_a) diff dist: 0.9894463764471134
# P(p_b > p_a) post samples: 0.98956

Вы предлагаете изменение в продукте $H$. 
Оно может улучшить продукт с вероятностью $h$ или ухудшить с вероятностью $1-h$.  
По итогам эксперимента вы решаете, оставлять изменение или нет.  
Вы можете ошибиться в решении: пропустить улучшение с вероятностью $\alpha$ или оставить ухудшение с вероятностью $1 - \beta$.  


P(Keep & Improve), P(Discard & Improve), P(Keep & Degrade), P(Discard & Degrade).


Вы предлагаете изменение в продукте $H$. 
Оно может улучшить продукт с вероятностью $h$ или ухудшить с вероятностью $1-h$.  
По итогам эксперимента вы решаете, оставлять изменение или нет.  
Вы можете пропустить улучшение с вероятностью $\alpha$ или оставить ухудшение с вероятностью $1 - \beta$.  
Запишите вероятность выбора оптимального варианта: P(Keep & Improve) + P(Discard & Degrade).  
Запишите вероятность оптимального действия с изменением - оставить улучшение или не внедрить ухудшение.

Изменение может улучшить продукт с вероятностью $h$ или ухудшить с вероятностью $1-h$.  
При выборе оставлять изменение или нет можно пропустить улучшение с вероятностью $a$ или внедрить ухудшение с вероятностью $1 - b$.  
Запишите вероятность "правильного выбора" - оставить улучшение или не внедрить ухудшение.


Всего провели $N$ экспериментов. В каждом эксперименте выбирали либо улучшающий, либо не ухудшающий вариант.
Посчитайте количество внедрений реальных улучшений. 
В ответе приведите значение для N = 50, a = 0.05, b = 0.2, h = 0.2.

$$
S = ? N \frac{h (1-a)}{h(1-a) + (1-h)b}
\\
S = N h (1-a)
$$

Ухудшения:

$$
L = (1-h) (1-b)
$$

$$
P(\text{правильный выбор}) = h (1-a) + (1-h) b
$$

Обратите внимание, что вероятность "правильного выбора" зависит от качества предлагаемых изменений. 
Если изменения чаще улучшают продукт, она будет ближе к $1-a$, если ухудшают - к $b$.

In [ ]:
N = 50
a = 0.05
#b = 0.2
b = a
h = 0.2
S = N * h * (1-a) / (h * (1-a) + (1-h)*b)
print('S', S)

S = N * h * (1-a)
print('S', S)

L = N * (1-h) * (1-b)
print('L', L)

Похожую ситуацию рассматривают в "проверках нулевых гипотез". Вместо улучшения и ухудшения формулируют гипотезу $H_0$.

<center>
<img src="../figs/bayes_square_h0.png" alt="bayes_square_h0" width="500"/>
</center>